In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./datasets/MNIST/train.csv')
test = pd.read_csv('./datasets/MNIST/test.csv')
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = train.drop(labels = ['label'], axis=1)
Y = train['label']
X.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Normalize

In [5]:
X = X/255.0
test = test/255.0

## Label Encoding

In [6]:
Y = tf.keras.utils.to_categorical(Y, num_classes=10)

## train-test split

In [7]:
random_seed = 0
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3, random_state=random_seed)

In [10]:
print(X_train.shape, Y_train.shape)
print(X_val.shape, Y_val.shape)

(29400, 784) (29400, 10)
(12600, 784) (12600, 10)


## Model

In [26]:
# CHECK : Constants
omega = 1.

class ELM(object):
    def __init__(self, sess, batch_size, input_len, hidden_num, output_len):
        '''
        Args:
          sess : TensorFlow session.
          batch_size : The batch size (N)
          input_len : The length of input. (L)
          hidden_num : The number of hidden node. (K)
          output_len : The length of output. (O)
        '''
    
        self._sess = sess 
        self._batch_size = batch_size
        self._input_len = input_len
        self._hidden_num = hidden_num
        self._output_len = output_len 

        # for train
        self._x0 = tf.placeholder(tf.float32, [self._batch_size, self._input_len])
        self._t0 = tf.placeholder(tf.float32, [self._batch_size, self._output_len])

        # for test
        self._x1 = tf.placeholder(tf.float32, [None, self._input_len])
        self._t1 = tf.placeholder(tf.float32, [None, self._output_len])

        self._W = tf.Variable(
          tf.random_normal([self._input_len, self._hidden_num]),
          trainable=False, dtype=tf.float32)
        self._b = tf.Variable(
          tf.random_normal([self._hidden_num]),
          trainable=False, dtype=tf.float32)
        self._beta = tf.Variable(
          tf.zeros([self._hidden_num, self._output_len]),
          trainable=False, dtype=tf.float32)
        self._var_list = [self._W, self._b, self._beta]

        self.H0 = tf.matmul(self._x0, self._W) + self._b # N x L
        self.H0_T = tf.transpose(self.H0)

        self.H1 = tf.matmul(self._x1, self._W) + self._b # N x L
        self.H1_T = tf.transpose(self.H1)

        # beta analytic solution : self._beta_s (K x O)
        if self._input_len < self._hidden_num: # L < K
            identity = tf.constant(np.identity(self._hidden_num), dtype=tf.float32)
            self._beta_s = tf.matmul(tf.matmul(tf.matrix_inverse(
                tf.matmul(self.H0_T, self.H0) + identity/omega), 
                self.H0_T), self._t0)
          # _beta_s = (H_T*H + I/om)^(-1)*H_T*T
        else:
            identity = tf.constant(np.identity(self._batch_size), dtype=tf.float32)
            self._beta_s = tf.matmul(tf.matmul(self.H0_T, tf.matrix_inverse(
                tf.matmul(self.H0, self.H0_T)+identity/omega)), self._t0)
          # _beta_s = H_T*(H*H_T + I/om)^(-1)*T

        self._assign_beta = self._beta.assign(self._beta_s)
        self._fx0 = tf.matmul(self.H0, self._beta)
        self._fx1 = tf.matmul(self.H1, self._beta)

        self._cost = tf.reduce_mean(tf.cast(tf.nn.softmax_cross_entropy_with_logits(logits = self._fx0, labels=self._t0), tf.float32))

        self._init = False
        self._feed = False

        # for the mnist test
        self._correct_prediction = tf.equal(tf.argmax(self._fx1,1), tf.argmax(self._t1,1))
        self._accuracy = tf.reduce_mean(tf.cast(self._correct_prediction, tf.float32))
        self._testcost = tf.reduce_mean(tf.cast(tf.nn.softmax_cross_entropy_with_logits(logits = self._fx1, labels=self._t1), tf.float32))

    def feed(self, x, t):
        '''
        Args :
          x : input array (N x L)
          t : label array (N x O)
        '''

        if not self._init : self.init()
        self._sess.run(self._assign_beta, {self._x0:x, self._t0:t})
        print(self._sess.run(self._cost, {self._x0:x, self._t0:t}))
        print(self._sess.run(self._W))
        print(self._sess.run(self._b))
        self._feed = True

    def init(self):
        self._sess.run(tf.initialize_variables(self._var_list))
        self._init = True

    def test(self, x, t=None):
        if not self._feed : exit("Not feed-forward trained")
        if t is not None :
            print("Accuracy: {:.9f}".format(self._sess.run(self._accuracy, {self._x1:x, self._t1:t})))
            print(self._sess.run(self._testcost, {self._x1:x, self._t1:t}))
        else :
            return self._sess.run(self._fx1, {self._x1:x})


In [27]:
# Basic tf setting
tf.set_random_seed(2016)
sess = tf.Session()

In [29]:
# Construct ELM
batch_size = 5000
hidden_num = 150
print("batch_size : {}".format(batch_size))
print("hidden_num : {}".format(hidden_num))
elm = ELM(sess, batch_size, 784, hidden_num, 10)

# one-step feed-forward training
train_x, train_y = (X_train[:batch_size], Y_train[:batch_size])
print(Y_train[:batch_size].shape)
elm.feed(train_x, train_y)



batch_size : 5000
hidden_num : 150
(5000, 10)
1.8510379
[[ 7.3176965e-02 -7.6991111e-02  6.6746467e-01 ...  7.7124542e-01
   6.2056031e-04 -9.4813362e-02]
 [ 4.7204405e-01  4.3908125e-01 -2.8025737e+00 ... -7.5376862e-01
   6.7355454e-01  1.5405505e+00]
 [-4.3485072e-01  7.4403179e-01 -1.0111090e+00 ... -1.9169935e+00
   6.7440051e-01  1.2137524e+00]
 ...
 [-8.0669880e-01  4.0632170e-01  2.4900800e-02 ...  6.5346867e-01
  -6.9536453e-01 -7.2745651e-01]
 [ 4.8483407e-01 -3.0736339e-01 -1.5599908e+00 ... -8.0401409e-01
  -1.1215825e+00 -5.9987509e-01]
 [-5.2899957e-02  4.1458067e-01  1.4560453e+00 ...  5.6286615e-01
   1.5148175e+00  8.0487639e-01]]
[-0.36052394 -0.802075    0.42311585 -0.9549258   0.12840179 -0.6886704
 -1.4160911  -1.6987478   0.06524988 -2.011726    0.8020627  -0.979798
 -0.24571842  0.6795765   0.7757317   0.9537463   1.2108722  -1.023122
  1.1804876  -1.0763868   0.08867537  0.28905237 -0.41358733 -0.37960243
  0.17790522  1.5864985   0.71536976  0.90141857 -0.20575

In [18]:
# testing
elm.test(X_val, Y_val)

Accuracy: 0.828174591
1.8621845


In [ ]:
Accuracy: 0.825079381
Accuracy: 0.825476170
[[-0.00024868  0.00246471 -0.00179496 ...  0.00219151  0.00024565
  -0.00116104]
 [-0.00109379  0.00472928 -0.00061047 ...  0.00013331  0.0021384
   0.00035015]
 [-0.00325735  0.00341256  0.00088452 ... -0.00226208  0.00226711
  -0.00151466]
 ...
 [ 0.00146626  0.00094969 -0.00018462 ...  0.0009436   0.00357442
  -0.00052654]
 [ 0.00237615  0.00077861  0.00184058 ... -0.00163503 -0.00084049
  -0.00311763]
 [-0.00246431 -0.00155569 -0.00293018 ... -0.00287577  0.00222375
  -0.0012665 ]]

In [ ]:
[[ 0.39634472  0.579398    1.1798518  ...  0.72396314  0.23257598
  -0.5066404 ]
 [ 1.5386133   0.01988766  0.5939701  ...  1.1362087   1.1128877
  -0.77877533]
 [ 1.9049335   1.0802777   0.9461669  ... -0.23150313 -0.29270864
   0.46897396]
 ...
 [-1.3471951  -1.1323553   0.51566595 ... -0.08844847  0.20875141
   1.5831931 ]
 [-0.45137528 -0.48304653  0.51466745 ... -1.3749615  -0.17955118
   0.13751285]
 [ 0.59962064 -0.08881681 -0.91595584 ...  0.42913738  0.11801394
  -2.5374403 ]]

In [ ]:
[ 1.8614435  -0.37014237 -0.01570693  1.7599701   0.2839093  -0.00864826
 -0.5884379  -1.4333824   1.5079826   0.44312927 -0.26730567  2.2609136
  2.5005858  -0.03939011  0.32231337  1.5867906  -1.1311699  -0.43755046
  0.07879981  0.13014644 -0.24867058 -1.2283027  -2.8281512   0.375159
  0.65426826  1.694165    0.1024062   0.70552015  0.30224052 -0.5512746
  0.38308257  0.00731312 -0.48388746 -0.6342199   0.31025523 -0.14976434
 -0.35945264 -1.1671903   0.12641531 -1.459952    1.9473269   0.19016044
 -1.5639586  -0.7317004   0.05978071  1.1778693   0.8650122  -0.28275567
 -0.03378246  1.0118301  -0.54881734  0.70299935 -0.6076311   0.4906861
  1.0951065   2.1433887   1.5572025   0.2922932  -1.0545242   0.40753537
 -0.39609653  0.26018584 -1.0178506  -0.24340983  0.5952991  -1.025846
  0.544627    0.26166058  0.61731    -0.4909846  -0.8437538  -0.9603444
  0.08755533 -0.4290903   0.7232904  -0.91362315  0.6547225   0.9290343
 -0.43424726 -0.10504516  0.6642769   0.9680431   1.0611808  -0.22828786
  1.1288342  -1.514031    0.00661617  0.5621307   0.5039861   0.84457994
  0.7361555   0.04215516 -0.5486878  -0.481361    0.4960944  -0.3697716
  0.4843361  -0.3877731  -1.4481905  -0.88229364 -1.8143269  -0.20859176
  0.8314413  -0.2074421   0.30476135 -0.5646082  -0.32877874  0.7394618
  0.63152456  0.48769003 -2.7870147   0.24948429  0.84909135 -0.5999759
  0.2783356   1.5617613  -1.4880936  -0.714584   -0.4650468   0.6691025
 -0.37828234 -0.6522176   0.20336439 -0.0559369   0.84059715  1.1137273
  1.0552956   2.0521653  -0.02903977 -0.6075414   1.7454621  -0.1528776
 -1.8107432   0.0430164  -1.0583352   1.239547   -0.46473277 -2.2136555
 -0.32691494  1.0632347   0.0664447   1.2535411  -0.15716933 -0.6045304
  1.542862   -0.4022672   0.93860453  0.6531672   0.49710882  1.0318404 ]